In [3]:
# Ref 1: https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/
# Ref 2: https://keras.io/getting-started/functional-api-guide/#multi-input-and-multi-output-models

import pandas as pd
import numpy as np

import cv2

import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

from os import path

In [4]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [5]:
def parse_file_name(filename):
    # training_0000_segment-10206293520369375008_2796_800_2816_800_with_camera_labels.csv
    if len(filename.split('.')) > 1:
        filename = filename.split('.')[0]
    parts = filename.split('_') 
    category = parts[0]           # training  
    tar = '_'.join(parts[0:2])    # training_0000
    segment = '_'.join(parts[2:]) # segment-10206293520369375008_2796_800_2816_800_with_camera_labels
    
    pred_path = '/home/zg2309/prediction/sslstm-image5_epoch_2000_hidden_128_observe_10_predict_5/' + tar + '/' + segment + '/prediction.npy'
    video_path = '/home/dataset/videos/' + category + '/' + tar + '/' + segment + '/video.mp4'
    data_path = '/home/dataset/team2/' + category + '/' + tar + '/' + filename + '.csv'
    out_path = '/home/dataset/visualization/12.5/' + '_'.join(parts[0:3]) + '.mp4'
    return {'pred_path': pred_path, 'video_path': video_path, 'data_path': data_path, 'out_path': out_path}

In [18]:
def visualize(filenames, model=''):
    for filename in filenames:
        # training_0000_segment-10206293520369375008_2796_800_2816_800_with_camera_labels.csv
        parsed = parse_file_name(filename)
        video_path = parsed['video_path']
        prediction_path = parsed['pred_path']
        truth_path = parsed['data_path']
        out_video_path = parsed['out_path']
        print(parsed)
        
        if not path.exists(video_path): 
            print("No Images! - " + video_path) 
            return 
        elif not path.exists(prediction_path): 
            print("No Predictions! - " + prediction_path) 
            return
        elif not path.exists(truth_path): 
            print("No Data! - " + truth_path) 
            return

        prediction = pd.DataFrame(np.load(prediction_path))
        ground_truth = pd.DataFrame(pd.read_csv(truth_path, header=0).values)
        cap = cv2.VideoCapture(video_path)
        
        dt = 0.1 # Scale for moving the vehicles
        frame_id = 0
        fig, axs = plt.subplots(2)
        fig.tight_layout()
        canvas = FigureCanvas(fig)
        fig.subplots_adjust(hspace=0.5)

        acc_pred_points = [[], [], []]
        acc_truth_points = [[], [], []]

        # For preview purpose.
        preview_shape = (288, 864) # Hard coded!
        car_size_2 = (10, 20)
        starting_point = (int(preview_shape[0] / 2), car_size_2[1])
        
        if (np.var(ground_truth.iloc[:, 0])) < (np.var(ground_truth.iloc[:, 1])):
            temp = ground_truth.iloc[:, 1].copy()
            ground_truth.iloc[:, 1] = ground_truth.iloc[:, 0]
            ground_truth.iloc[:, 0] = temp
        if np.var(ground_truth.iloc[:, -3]) < np.var(ground_truth.iloc[:, -2]):
            temp = ground_truth.iloc[:, -3].copy()
            ground_truth.iloc[:, -3] = ground_truth.iloc[:, -2]
            ground_truth.iloc[:, -2] = temp
        if np.var(prediction.iloc[:, 0]) < np.var(ground_truth.iloc[:, 1]):
            temp = prediction.iloc[:, 0].copy()
            prediction.iloc[:, 0] = prediction.iloc[:, 1]
            prediction.iloc[:, 1] = temp
        
        # Retrieved from data.
        v_truth = -np.array(ground_truth.iloc[0, 0:2].copy().tolist())
        v_pred = -np.array(ground_truth.iloc[0, 0:2].copy().tolist())

        acc_truth = [0.0, 0.0]
        acc_pred = [0.0, 0.0]

        pos_truth = [0, starting_point[1], ]
        pos_pred = [0, starting_point[1], ]
        pos_front = []

        out = None
        
        # Find the maximum absolute value for axis limitations.
        lim_x_truth = max(abs(ground_truth.iloc[:, -3].max()), abs(ground_truth.iloc[:, -3].min()))
        lim_y_truth = max(abs(ground_truth.iloc[:, -2].max()), abs(ground_truth.iloc[:, -2].min()))
        lim_x_pred = max(abs(prediction.iloc[:, 0].max()), abs(prediction.iloc[:, 1].min()))
        lim_y_pred = max(abs(prediction.iloc[:, 0].max()), abs(prediction.iloc[:, 1].min()))
        lim_acc = max(lim_x_truth, lim_y_truth, lim_x_pred, lim_y_pred)

        # Read until video is completed
        while(cap.isOpened() and frame_id < len(prediction) and frame_id < len(ground_truth)):
            # Capture frame-by-frame
            ret, frame = cap.read()
            if ret == True:
                frame_h, frame_w, frame_c = frame.shape

                pos_truth = [(pos_truth[i] + v_truth[1 - i] * dt) for i in range(2)]
                v_truth = -np.array(ground_truth.iloc[frame_id, 0:2].tolist())
                    
                d_front = ground_truth.iloc[frame_id, 3:5].tolist()[::-1]
                pos_front = [(pos_truth[i] + d_front[i]) for i in range(2)]

                pos_pred = [(pos_pred[i] + v_pred[1 - i] * dt) for i in range(2)]
                # Caution: SIGN!!!
                v_pred = [(v_pred[i] - acc_pred[i] * dt) for i in range(2)]
                # v_pred[0] = 0.0 if abs(v_pred[0]) < 2.0 else v_pred[0]
                # v_pred[1] = 0.0 if abs(v_pred[1]) < 0.02 else v_pred[1]

                acc_pred = (prediction.iloc[frame_id, 0:2]).tolist()
                acc_pred[0] *= 1.4 if acc_pred[0] > 0 else 1.0
                acc_truth = ground_truth.iloc[frame_id, -3:-1].tolist()

                for i in range(2):
                    # Caution: SIGN!!!
                    acc_pred_points[i].append(-acc_pred[i])
                    acc_truth_points[i].append(-acc_truth[i])

                axs[0].set_title("x acc")
                axs[1].set_title("y acc")
                for i in range(2):
                    axs[i].set_xlim(0, 200)
                    margin = 0.2
                    axs[i].set_ylim(-lim_acc * (1 + margin), lim_acc * (1 + margin))
                    axs[i].hlines(0, 0, 200, '0.5', lw=1, linestyles='dashed')
                    axs[i].plot(acc_pred_points[i], label="Prediction")
                    axs[i].plot(acc_truth_points[i], label="Ground Truth")
                    axs[i].legend(loc='lower right')

                canvas.draw()       # draw the canvas, cache the renderer
                fig_w, fig_h = fig.get_size_inches() * fig.get_dpi() 
                fig_w, fig_h = (int(fig_w), int(fig_h))
                frame = image_resize(frame, height=fig_h)
                image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(fig_h, fig_w, 3)

                combined = cv2.hconcat([image, frame])
                display_scale = (0.5, 6)
                pred_preview = np.zeros((combined.shape[0], combined.shape[1], 3), np.uint8)
                cv2.line(pred_preview, (0, starting_point[0]),
                         (int(pred_preview.shape[1]), starting_point[0]), (100, 100, 100), 1)
                cv2.line(pred_preview, (display_scale[1] * starting_point[1], 0), 
                         (display_scale[1] * starting_point[1], int(pred_preview.shape[0])), (100, 200, 100), 1)
                cv2.rectangle(pred_preview, 
                              (int(display_scale[1] * pos_truth[1]) - car_size_2[1], 
                               int(display_scale[0] * pos_truth[0]) - car_size_2[0] + int(preview_shape[0] / 2)), 
                              (int(display_scale[1] * pos_truth[1]) + car_size_2[1], 
                               int(display_scale[0] * pos_truth[0]) + car_size_2[0] + int(preview_shape[0] / 2)), 
                              (200,0,0), cv2.FILLED)
                cv2.rectangle(pred_preview, 
                              (int(display_scale[1] * pos_pred[1]) - car_size_2[1], 
                               int(display_scale[0] * pos_pred[0]) - car_size_2[0] + int(preview_shape[0] / 2)), 
                              (int(display_scale[1] * pos_pred[1]) + car_size_2[1], 
                               int(display_scale[0] * pos_pred[0]) + car_size_2[0] + int(preview_shape[0] / 2)), 
                              (0,155,232), 2)
                cv2.rectangle(pred_preview, 
                              (int(display_scale[1] * pos_front[1]) - car_size_2[1], 
                               int(display_scale[0] * pos_front[0]) - car_size_2[0] + int(preview_shape[0] / 2)), 
                              (int(display_scale[1] * pos_front[1]) + car_size_2[1], 
                               int(display_scale[0] * pos_front[0]) + car_size_2[0] + int(preview_shape[0] / 2)), 
                              (0,0,255), 2)

                combined = cv2.vconcat([combined, pred_preview])
                comb_shape = combined.shape
                if out is None:
                    out = cv2.VideoWriter(out_video_path, cv2.VideoWriter_fourcc(*'XVID'), 10, (comb_shape[1], comb_shape[0]))
                out.write(combined)

                canvas.renderer.clear()
                for i in range(2):
                    axs[i].clear()
                if frame_id % 20 == 0:
                    print("[%s]: %.2f%% finished" % (filename, frame_id / 198.0 * 100.0))

                frame_id += 1
        # When everything done, release the video capture object
        plt.close(fig)
        out.release()
        
        print("[%s] finished" % (filename))
        

In [62]:
visualize(['validation_0001_segment-10289507859301986274_4200_000_4220_000'])

{'pred_path': '/home/zg2309/prediction/sslstm-image5_epoch_2000_hidden_128_observe_10_predict_5/validation_0001/segment-10289507859301986274_4200_000_4220_000/prediction.npy', 'video_path': '/home/dataset/videos/validation/validation_0001/segment-10289507859301986274_4200_000_4220_000/video.mp4', 'out_path': '/home/dataset/visualization/12.5/validation_0001_segment-10289507859301986274.mp4', 'data_path': '/home/dataset/team2/validation/validation_0001/validation_0001_segment-10289507859301986274_4200_000_4220_000.csv'}
[validation_0001_segment-10289507859301986274_4200_000_4220_000]: 0.00% finished
[validation_0001_segment-10289507859301986274_4200_000_4220_000]: 10.10% finished
[validation_0001_segment-10289507859301986274_4200_000_4220_000]: 20.20% finished
[validation_0001_segment-10289507859301986274_4200_000_4220_000]: 30.30% finished
[validation_0001_segment-10289507859301986274_4200_000_4220_000]: 40.40% finished
[validation_0001_segment-10289507859301986274_4200_000_4220_000]: 

In [19]:
visualize(['validation_0000_segment-11660186733224028707_420_000_440_000_with_camera_labels',
    'validation_0000_segment-13178092897340078601_5118_604_5138_604_with_camera_labels',
    'validation_0000_segment-14956919859981065721_1759_980_1779_980_with_camera_labels',
    'validation_0000_segment-16213317953898915772_1597_170_1617_170_with_camera_labels',
    'validation_0000_segment-6001094526418694294_4609_470_4629_470_with_camera_labels',
    'validation_0001_segment-10289507859301986274_4200_000_4220_000',
    'validation_0001_segment-10335539493577748957_1372_870_1392_870',
    'validation_0001_segment-10359308928573410754_720_000_740_000',
    'validation_0001_segment-10868756386479184868_3000_000_3020_000',
    'validation_0001_segment-11450298750351730790_1431_750_1451_750',
    'validation_0001_segment-12374656037744638388_1412_711_1432_711',
    'validation_0001_segment-12496433400137459534_120_000_140_000',
    'validation_0002_segment-12866817684252793621_480_000_500_000',
    'validation_0002_segment-12940710315541930162_2660_000_2680_000',
    'validation_0002_segment-13184115878756336167_1354_000_1374_000',
    'validation_0002_segment-13299463771883949918_4240_000_4260_000',
    'validation_0002_segment-13694146168933185611_800_000_820_000',
    'validation_0002_segment-13982731384839979987_1680_000_1700_000',
    'validation_0002_segment-14165166478774180053_1786_000_1806_000',
    'validation_0002_segment-14262448332225315249_1280_000_1300_000',
    'validation_0002_segment-14486517341017504003_3406_349_3426_349',
    'validation_0002_segment-1457696187335927618_595_027_615_027',
    'validation_0003_segment-14931160836268555821_5778_870_5798_870',
    'validation_0003_segment-1505698981571943321_1186_773_1206_773',
    'validation_0003_segment-16229547658178627464_380_000_400_000',
    'validation_0003_segment-17065833287841703_2980_000_3000_000',
    'validation_0004_segment-17152649515605309595_3440_000_3460_000',
    'validation_0004_segment-17694030326265859208_2340_000_2360_000',
    'validation_0004_segment-17763730878219536361_3144_635_3164_635',
    'validation_0004_segment-17860546506509760757_6040_000_6060_000',
    'validation_0004_segment-18446264979321894359_3700_000_3720_000',
    'validation_0004_segment-1943605865180232897_680_000_700_000',
    'validation_0005_segment-2551868399007287341_3100_000_3120_000',
    'validation_0005_segment-3731719923709458059_1540_000_1560_000',
    'validation_0005_segment-4575389405178805994_4900_000_4920_000'])

{'data_path': '/home/dataset/team2/validation/validation_0000/validation_0000_segment-11660186733224028707_420_000_440_000_with_camera_labels.csv', 'video_path': '/home/dataset/videos/validation/validation_0000/segment-11660186733224028707_420_000_440_000_with_camera_labels/video.mp4', 'pred_path': '/home/zg2309/prediction/sslstm-image5_epoch_2000_hidden_128_observe_10_predict_5/validation_0000/segment-11660186733224028707_420_000_440_000_with_camera_labels/prediction.npy', 'out_path': '/home/dataset/visualization/12.5/validation_0000_segment-11660186733224028707.mp4'}
[validation_0000_segment-11660186733224028707_420_000_440_000_with_camera_labels]: 0.00% finished
[validation_0000_segment-11660186733224028707_420_000_440_000_with_camera_labels]: 10.10% finished
[validation_0000_segment-11660186733224028707_420_000_440_000_with_camera_labels]: 20.20% finished
[validation_0000_segment-11660186733224028707_420_000_440_000_with_camera_labels]: 30.30% finished
[validation_0000_segment-1166

[validation_0000_segment-6001094526418694294_4609_470_4629_470_with_camera_labels] finished
{'data_path': '/home/dataset/team2/validation/validation_0001/validation_0001_segment-10289507859301986274_4200_000_4220_000.csv', 'video_path': '/home/dataset/videos/validation/validation_0001/segment-10289507859301986274_4200_000_4220_000/video.mp4', 'pred_path': '/home/zg2309/prediction/sslstm-image5_epoch_2000_hidden_128_observe_10_predict_5/validation_0001/segment-10289507859301986274_4200_000_4220_000/prediction.npy', 'out_path': '/home/dataset/visualization/12.5/validation_0001_segment-10289507859301986274.mp4'}
[validation_0001_segment-10289507859301986274_4200_000_4220_000]: 0.00% finished
[validation_0001_segment-10289507859301986274_4200_000_4220_000]: 10.10% finished
[validation_0001_segment-10289507859301986274_4200_000_4220_000]: 20.20% finished
[validation_0001_segment-10289507859301986274_4200_000_4220_000]: 30.30% finished
[validation_0001_segment-10289507859301986274_4200_000_4

[validation_0001_segment-12374656037744638388_1412_711_1432_711]: 70.71% finished
[validation_0001_segment-12374656037744638388_1412_711_1432_711]: 80.81% finished
[validation_0001_segment-12374656037744638388_1412_711_1432_711]: 90.91% finished
[validation_0001_segment-12374656037744638388_1412_711_1432_711] finished
{'data_path': '/home/dataset/team2/validation/validation_0001/validation_0001_segment-12496433400137459534_120_000_140_000.csv', 'video_path': '/home/dataset/videos/validation/validation_0001/segment-12496433400137459534_120_000_140_000/video.mp4', 'pred_path': '/home/zg2309/prediction/sslstm-image5_epoch_2000_hidden_128_observe_10_predict_5/validation_0001/segment-12496433400137459534_120_000_140_000/prediction.npy', 'out_path': '/home/dataset/visualization/12.5/validation_0001_segment-12496433400137459534.mp4'}
[validation_0001_segment-12496433400137459534_120_000_140_000]: 0.00% finished
[validation_0001_segment-12496433400137459534_120_000_140_000]: 10.10% finished
[v

[validation_0002_segment-13694146168933185611_800_000_820_000]: 50.51% finished
[validation_0002_segment-13694146168933185611_800_000_820_000]: 60.61% finished
[validation_0002_segment-13694146168933185611_800_000_820_000]: 70.71% finished
[validation_0002_segment-13694146168933185611_800_000_820_000]: 80.81% finished
[validation_0002_segment-13694146168933185611_800_000_820_000]: 90.91% finished
[validation_0002_segment-13694146168933185611_800_000_820_000] finished
{'data_path': '/home/dataset/team2/validation/validation_0002/validation_0002_segment-13982731384839979987_1680_000_1700_000.csv', 'video_path': '/home/dataset/videos/validation/validation_0002/segment-13982731384839979987_1680_000_1700_000/video.mp4', 'pred_path': '/home/zg2309/prediction/sslstm-image5_epoch_2000_hidden_128_observe_10_predict_5/validation_0002/segment-13982731384839979987_1680_000_1700_000/prediction.npy', 'out_path': '/home/dataset/visualization/12.5/validation_0002_segment-13982731384839979987.mp4'}
[va

[validation_0003_segment-14931160836268555821_5778_870_5798_870]: 20.20% finished
[validation_0003_segment-14931160836268555821_5778_870_5798_870]: 30.30% finished
[validation_0003_segment-14931160836268555821_5778_870_5798_870]: 40.40% finished
[validation_0003_segment-14931160836268555821_5778_870_5798_870]: 50.51% finished
[validation_0003_segment-14931160836268555821_5778_870_5798_870]: 60.61% finished
[validation_0003_segment-14931160836268555821_5778_870_5798_870]: 70.71% finished
[validation_0003_segment-14931160836268555821_5778_870_5798_870]: 80.81% finished
[validation_0003_segment-14931160836268555821_5778_870_5798_870]: 90.91% finished
[validation_0003_segment-14931160836268555821_5778_870_5798_870] finished
{'data_path': '/home/dataset/team2/validation/validation_0003/validation_0003_segment-1505698981571943321_1186_773_1206_773.csv', 'video_path': '/home/dataset/videos/validation/validation_0003/segment-1505698981571943321_1186_773_1206_773/video.mp4', 'pred_path': '/home

[validation_0004_segment-17763730878219536361_3144_635_3164_635]: 0.00% finished
[validation_0004_segment-17763730878219536361_3144_635_3164_635]: 10.10% finished
[validation_0004_segment-17763730878219536361_3144_635_3164_635]: 20.20% finished
[validation_0004_segment-17763730878219536361_3144_635_3164_635]: 30.30% finished
[validation_0004_segment-17763730878219536361_3144_635_3164_635]: 40.40% finished
[validation_0004_segment-17763730878219536361_3144_635_3164_635]: 50.51% finished
[validation_0004_segment-17763730878219536361_3144_635_3164_635]: 60.61% finished
[validation_0004_segment-17763730878219536361_3144_635_3164_635]: 70.71% finished
[validation_0004_segment-17763730878219536361_3144_635_3164_635]: 80.81% finished
[validation_0004_segment-17763730878219536361_3144_635_3164_635]: 90.91% finished
[validation_0004_segment-17763730878219536361_3144_635_3164_635] finished
{'data_path': '/home/dataset/team2/validation/validation_0004/validation_0004_segment-17860546506509760757_

[validation_0005_segment-4575389405178805994_4900_000_4920_000]: 0.00% finished
[validation_0005_segment-4575389405178805994_4900_000_4920_000]: 10.10% finished
[validation_0005_segment-4575389405178805994_4900_000_4920_000]: 20.20% finished
[validation_0005_segment-4575389405178805994_4900_000_4920_000]: 30.30% finished
[validation_0005_segment-4575389405178805994_4900_000_4920_000]: 40.40% finished
[validation_0005_segment-4575389405178805994_4900_000_4920_000]: 50.51% finished
[validation_0005_segment-4575389405178805994_4900_000_4920_000]: 60.61% finished
[validation_0005_segment-4575389405178805994_4900_000_4920_000]: 70.71% finished
[validation_0005_segment-4575389405178805994_4900_000_4920_000]: 80.81% finished
[validation_0005_segment-4575389405178805994_4900_000_4920_000]: 90.91% finished
[validation_0005_segment-4575389405178805994_4900_000_4920_000] finished


In [ ]:
import glob

test_folders = glob.glob('/home/zg2309/result/validation_0000/*/')
test_num = len(test_folders)
print('test folders num:', len(test_folders))


model_name = 'double_lstm_600_smooth'

visualize(test_folders, model=model_name)

test folders num: 25
Successfully opened video!
[/home/zg2309/result/validation_0000/segment-16751706457322889693_4475_240_4495_240_with_camera_labels/] finished
Successfully opened video!
[/home/zg2309/result/validation_0000/segment-10689101165701914459_2072_300_2092_300_with_camera_labels/] finished
Successfully opened video!
[/home/zg2309/result/validation_0000/segment-2094681306939952000_2972_300_2992_300_with_camera_labels/] finished
Successfully opened video!
[/home/zg2309/result/validation_0000/segment-13178092897340078601_5118_604_5138_604_with_camera_labels/] finished
Successfully opened video!
[/home/zg2309/result/validation_0000/segment-10448102132863604198_472_000_492_000_with_camera_labels/] finished
Successfully opened video!
[/home/zg2309/result/validation_0000/segment-11037651371539287009_77_670_97_670_with_camera_labels/] finished
Successfully opened video!
[/home/zg2309/result/validation_0000/segment-17612470202990834368_2800_000_2820_000_with_camera_labels/] finished